In [1]:
import h5py
import ast
from utils.misc_utils import get_logger

import numpy as np
from scipy import stats
from scipy.special import gamma, digamma
from scipy.integrate import nquad

In [2]:
from utils.bin_evaluators import Evaluator_BIN

eval = Evaluator_BIN()

eval.data_path = "data.hdf5"
eval.out_path = "results/bin.hdf5"
eval.logger = get_logger("results/bin_mi.log")

eval.quantity = "mi"

eval.hyper_params = ["scott", "fd", "sturges"]
eval.sample_sizes = [100, 200, 500, 1_000, 5_000, 10_000, 50_000, 100_000]
eval.seeds = range(1, 4)

for k, v in vars(eval).items():
    print(f"{k} - {v}")


eval.create_database()
eval.create_group()

experiments = [
    "bivariate-normal",
    "bivariate-normal-mixture",
    "gexp",
    "4d-gaussian"
]

data_path - data.hdf5
out_path - results/bin.hdf5
logger - <RootLogger root (DEBUG)>
quantity - mi
hyper_params - ['scott', 'fd', 'sturges']
sample_sizes - [100, 200, 500, 1000, 5000, 10000, 50000, 100000]
seeds - range(1, 4)
results - None


In [3]:
# # # # # BIVARIATE NORMAL # # # # #

experiment = "bivariate-normal"

# Calculate Truth
with h5py.File(eval.data_path, "r") as f:
    dist_params = ast.literal_eval(f[experiment]["p"].attrs["hyper_params"])

cov = np.array(dist_params[0][1])
d = len(cov)
true_mi = 0.5 * np.log(cov[0, 0] * cov[-1, -1] / np.linalg.det(cov)) # Reference

eval.evaluate_mutual_information(experiment, "scott")

# Save
eval.write_single_to_hdf5(experiment, true_mi)
print(f"True mutual information: {true_mi:.3f} nats")

2023-11-13 12:50:03 - Creating converter from 3 to 5
2023-11-13 12:50:03 - (BIVARIATE-NORMAL, scott, 10000, 1) - Time: 0.00371 s - Est.: 0.210 nats
2023-11-13 12:50:03 - (BIVARIATE-NORMAL, scott, 10000, 2) - Time: 0.00332 s - Est.: 0.217 nats
2023-11-13 12:50:03 - (BIVARIATE-NORMAL, scott, 10000, 3) - Time: 0.00338 s - Est.: 0.206 nats
2023-11-13 12:50:03 - Creating converter from 5 to 3


True mutual information: 0.144 nats


In [4]:
# # # # # BIVARIATE-NORMAL-MIXTURE # # # # #

experiment = "bivariate-normal-mixture"

# Calculate Truth
with h5py.File(eval.data_path, "r") as f:
    dist_params = ast.literal_eval(f[experiment]["p"].attrs["hyper_params"])

def pdf_normal(x, params):
    y = 0.0
    for dist in params:
        l, s, w = dist
        y += stats.norm(loc=l, scale=s).pdf(x) * w
    return y

def pdf_mnorm(x, y, params):
    z = 0.0
    for dist in params:
        l, s, w = dist
        z += stats.multivariate_normal(mean=l, cov=s).pdf(np.dstack((x, y))) * w
    return z

def mi_mnorm(x, y, params1):
    params_x = []
    params_y = []
    for dist in params1:
        params_x.append([dist[0][0], dist[1][0][0], dist[2]])
        params_y.append([dist[0][1], dist[1][1][1], dist[2]])

    pxy = pdf_mnorm(x, y, params1)
    px = pdf_normal(x, params_x)
    py = pdf_normal(y, params_y)
    
    return pxy * np.log(pxy / (px * py))


binorm_lims = [[-7, 7], [-7, 7]]

true_mi = nquad(mi_mnorm, binorm_lims, args=(dist_params,))[0] # Numerical Integration Result

eval.evaluate_mutual_information(experiment, "scott")

# Save
eval.write_single_to_hdf5(experiment, true_mi)
print(f"True mutual information: {true_mi:.3f} nats")

2023-11-13 12:51:01 - (BIVARIATE-NORMAL-MIXTURE, scott, 10000, 1) - Time: 0.00457 s - Est.: 0.191 nats
2023-11-13 12:51:01 - (BIVARIATE-NORMAL-MIXTURE, scott, 10000, 2) - Time: 0.00494 s - Est.: 0.188 nats
2023-11-13 12:51:01 - (BIVARIATE-NORMAL-MIXTURE, scott, 10000, 3) - Time: 0.00518 s - Est.: 0.181 nats


True mutual information: 0.140 nats


In [5]:
# # # # # GAMMA-EXPONENTIAL # # # # #

experiment = "gexp"

# Calculate Truth
with h5py.File(eval.data_path, "r") as f:
    dist_params = ast.literal_eval(f[experiment]["p"].attrs["hyper_params"])

tetha = dist_params[0][0]
true_mi = digamma(tetha) - np.log(tetha) + (1 / tetha) # Reference

eval.evaluate_mutual_information(experiment, "scott")

# Save
eval.write_single_to_hdf5(experiment, true_mi)
print(f"True mutual information: {true_mi:.3f} nats")

2023-11-13 12:51:04 - (GEXP, scott, 10000, 1) - Time: 0.00408 s - Est.: 0.191 nats
2023-11-13 12:51:04 - (GEXP, scott, 10000, 2) - Time: 0.00389 s - Est.: 0.202 nats
2023-11-13 12:51:05 - (GEXP, scott, 10000, 3) - Time: 0.00371 s - Est.: 0.187 nats


True mutual information: 0.158 nats


In [6]:
# # # # # 4D GAUSSIAN # # # # #

experiment = "4d-gaussian"
eval.seeds = range(1, 2)

# Calculate Truth
with h5py.File(eval.data_path, "r") as f:
    dist_params = ast.literal_eval(f[experiment]["p"].attrs["hyper_params"])

cov = np.array(dist_params[0][1])
d = len(cov)
true_mi = 0.5 * np.log(np.linalg.det(cov[:d-1,:d-1]) * cov[-1, -1] / np.linalg.det(cov)) # Reference

eval.evaluate_mutual_information(experiment, "scott")

# Save
eval.write_single_to_hdf5(experiment, true_mi)
print(f"True mutual information: {true_mi:.3f} nats")

2023-11-13 12:51:13 - (4D-GAUSSIAN, scott, 10000, 1) - Time: 2.42289 s - Est.: 2.645 nats


True mutual information: 0.235 nats
